In [1]:
import os
import numpy as np
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

def load_and_validate_data(file_path):
    """
    Load data from a file and validate its presence.
    Return None if data is invalid or shapes are inconsistent.
    """
    try:
        loaded_data = np.load(file_path)
        
        
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
       
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Apply a bandpass filter to the data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def preprocess_signal(signal, fs):
    """
    Preprocess the signal by bandpass filtering and normalization.
    """
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - np.mean(filtered_signal)) / np.std(filtered_signal)
    return normalized_signal
    
def combine_data_from_folder(folder_path , batch_size=100):
    """
    Combine and preprocess data from all valid files in the folder and generate VG images.
    """
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []
            

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            


train_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Train_data'
val_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Validation_data'
test_dir = 'C:\\Users\\nihal\\Desktop\\NIHAL_IMP_DOCS\\Internship_PPG\\Test_data'



In [2]:
val_data_generator = combine_data_from_folder(test_dir, batch_size=100)

In [3]:
import numpy as np
from skimage.transform import resize
import matplotlib.pyplot as plt

def generate_poincare_matrix(time_series, matrix_size=(224, 224), bins=224):
    x = time_series[:-1]
    y = time_series[1:]
    
    # Generate a 2D histogram (density plot) to represent the Poincaré plot as a matrix
    H, xedges, yedges = np.histogram2d(x, y, bins=bins, range=[[np.min(time_series), np.max(time_series)], [np.min(time_series), np.max(time_series)]])
    
    #Normalize the matrix values (optional)
    #H = H / np.max(H)  # Normalize to values between 0 and 1
    
    # Resize the matrix to the desired size (if necessary)
    if matrix_size != (bins, bins):
        H_resized = np.resize(H, matrix_size)
    else:
        H_resized = H
    
    return H_resized

In [4]:
import numpy as np
import os

output_dir = 'poincare_images_filt_batches'
os.makedirs(output_dir, exist_ok=True)

for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(val_data_generator):
   
    output_file = os.path.join(output_dir, f'filt_poi_batch_{batch_idx + 1}.npz')
    if os.path.exists(output_file):
        print(f"Batch {batch_idx + 1} already processed. Skipping...")
        continue
    print(f"Processing Batch {batch_idx + 1}...")
    print(len(combined_seg_dbp_batch))
    poi_all_fullpeaks = []
    for ppg in combined_ppg_batch:
        poi_matrix=generate_poincare_matrix(ppg.flatten())
        poi_all_fullpeaks.append(np.array(poi_matrix))
    
    poi_all_fullpeaks = np.array(poi_all_fullpeaks)
    
    batch_filename = os.path.join(output_dir, f'filt_poi_batch_{batch_idx + 1}.npz')
    np.savez_compressed(batch_filename, poi_images=poi_all_fullpeaks)
    
    print(f"Batch {batch_idx + 1} processing complete.")




Processing Batch 1...
916
Batch 1 processing complete.
Processing Batch 2...
492
Batch 2 processing complete.
Processing Batch 3...
120
Batch 3 processing complete.
Processing Batch 4...
1003
Batch 4 processing complete.
Processing Batch 5...
454
Batch 5 processing complete.
Processing Batch 6...
135
Batch 6 processing complete.
Processing Batch 7...
522
Batch 7 processing complete.
Processing Batch 8...
549
Batch 8 processing complete.
Processing Batch 9...
993
Batch 9 processing complete.
Processing Batch 10...
434
Batch 10 processing complete.
Processing Batch 11...
525
Batch 11 processing complete.
Processing Batch 12...
1291
Batch 12 processing complete.
Processing Batch 13...
537
Batch 13 processing complete.
Processing Batch 14...
308
Batch 14 processing complete.
Processing Batch 15...
918
Batch 15 processing complete.
Processing Batch 16...
378
Batch 16 processing complete.
Processing Batch 17...
437
Batch 17 processing complete.
Processing Batch 18...
831
Batch 18 processing 

KeyboardInterrupt: 

In [4]:
def load_poi_images(folder_path, batch_idx):
    poi_images_path = os.path.join(folder_path, f'filt_poi_batch_{batch_idx}.npz')
    
    if not os.path.exists(poi_images_path):
        print(f"poi image files do not exist for Batch {batch_idx}.")
        return None
    
    poi_images = np.load(poi_images_path)['poi_images']
    
    return poi_images

folder_path = r'C:\Users\nihal\Desktop\poincare_images_filt_batches'

def plot_poincare_matrix(poincare_matrix):
    plt.figure(figsize=(6, 6))
    plt.imshow(poincare_matrix, cmap='hot', origin='lower')  # You can change the colormap
    plt.colorbar()  # Add color bar to show the value scale
    plt.title("Poincaré Plot Matrix")
    plt.xlabel("x[i] (current point)")
    plt.ylabel("x[i+1] (next point)")
    plt.show()

In [5]:

all_poi_images = []
all_sbp_values = []
all_dbp_values = []


for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(val_data_generator):
   
    #if batch_idx + 1>4 :
        #break
    print(f"Processing Batch {batch_idx + 1}...")
    poi_images = load_poi_images(folder_path,batch_idx + 1)
    if poi_images is None:
        print(f"No poincare images found for Batch {batch_idx + 1}. Stopping...")
        break
    
    
    
    all_poi_images.append(np.array(poi_images))
    all_sbp_values.append(np.array(combined_seg_sbp_batch.flatten()))
    all_dbp_values.append(np.array(combined_seg_dbp_batch.flatten()))

    print(f"Batch {batch_idx + 1} processing complete.")
    

all_poi_images = np.concatenate(all_poi_images, axis=0)
all_sbp_values = np.concatenate(all_sbp_values, axis=0)
all_dbp_values = np.concatenate(all_dbp_values, axis=0)     

Processing Batch 1...
Batch 1 processing complete.
Processing Batch 2...
Batch 2 processing complete.
Processing Batch 3...
Batch 3 processing complete.
Processing Batch 4...
Batch 4 processing complete.
Processing Batch 5...


MemoryError: Unable to allocate 174. MiB for an array with shape (22779904,) and data type float64

In [ ]:
y = np.column_stack((all_sbp_values, all_dbp_values))
poi_images_normal = np.array(all_poi_images)

In [ ]:
plot_poincare_matrix(poi_images_normal[0])

In [ ]:
len(poi_images_normal)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import Ridge


batch_size = 32


def image_generator(images, labels, batch_size):
    num_samples = images.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]
            batch_images_reshaped = batch_images.reshape(-1, 224, 224, 1)
            batch_images_rgb = np.concatenate([batch_images_reshaped] * 3, axis=-1)
            batch_images_preprocessed = preprocess_input(batch_images_rgb)
            yield batch_images_preprocessed, batch_labels


In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import preprocess_input


base_model_mobile = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model_mobile.layers:
    layer.trainable = False


x = base_model_mobile.output
x = Flatten(name='flatten')(x)
output = Dense(512, activation='linear', name='fc1')(x)


# Define model
model_mobile = Model(inputs=base_model_mobile.input, outputs=output)


model_mobile.summary()


In [ ]:
images = poi_images_normal

In [ ]:
labels = y
num_samples = images.shape[0]

In [ ]:
# Define a new model to extract features from the 'fc1' layer of the trained model
feature_extractor_mobile = Model(inputs=model_mobile.input, outputs=model_mobile.get_layer('fc1').output)

# Function to extract features
def extract_features(generator, num_samples):
    features = []
    for batch_images, _ in generator:
        batch_features = feature_extractor_mobile.predict(batch_images)
        features.append(batch_features)
        if len(features) * batch_size >= num_samples:
            break
    return np.vstack(features)


feature_generator_normal_1 = image_generator(images, np.zeros((num_samples,)), batch_size)


features_normal_1 = extract_features(feature_generator_normal_1, num_samples)

In [ ]:
features_normal_flattened_1= features_normal_1.reshape(features_normal_1.shape[0],-1)


features_1= features_normal_flattened_1

In [ ]:
from sklearn.model_selection import train_test_split

Mobile_train,Mobile_test,label_train,label_test=train_test_split(features_1,labels, test_size=0.2, random_state=42)
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(Mobile_train, label_train)


predicted_mobile = ridge_model.predict(Mobile_test)

In [ ]:
predicted_sbp =predicted_mobile[:, 0]  
predicted_dbp =predicted_mobile[:, 1]  

true_sbp = label_test[:, 0]  
true_dbp = label_test[:, 1]  

def calculate_metrics(predicted, true):
    
    R = np.corrcoef(predicted, true)[0, 1]

   
    MAE = np.mean(np.abs(predicted - true))

   
    RMSE = np.sqrt(np.mean((predicted - true) ** 2))

    
    ME = np.mean(predicted - true)
    SD = np.std(predicted - true)

    return R, MAE, RMSE, (ME, SD)


sbp_metrics = calculate_metrics(predicted_sbp, true_sbp)
print(f"SBP Metrics: R={sbp_metrics[0]}, MAE={sbp_metrics[1]}, RMSE={sbp_metrics[2]}, ME±SD={sbp_metrics[3]}")


dbp_metrics = calculate_metrics(predicted_dbp, true_dbp)
print(f"DBP Metrics: R={dbp_metrics[0]}, MAE={dbp_metrics[1]}, RMSE={dbp_metrics[2]}, ME±SD={dbp_metrics[3]}")